In [1]:
import yt_dlp
from pytubefix import Search
import whisper
import torch
import os
import re

In [2]:
def format_time_to_srt(seconds):
    """Helper function to convert time in seconds to SRT format (HH:MM:SS,MS)"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"

In [3]:
def transcribe_with_whisper(audio_path, model, language="en"):
    # Transcription function with SRT output
    result = model.transcribe(
        audio_path,
        language=language,  # Language specified in global variable
        task="transcribe",
        fp16=torch.cuda.is_available()  # Use float16 if on GPU
    )

    # Extract the segments with timestamps and text
    segments = result["segments"]

    # Prepare the SRT formatted transcription
    srt_transcription = ""
    for i, segment in enumerate(segments, 1):
        start_time = segment["start"]
        end_time = segment["end"]
        text = segment["text"]
        
        # Convert times from seconds to SRT format (HH:MM:SS,MS)
        start_time_srt = format_time_to_srt(start_time)
        end_time_srt = format_time_to_srt(end_time)
        
        # Append each segment in SRT format (ensuring correct spacing)
        print(text)
        srt_transcription += f"{i}\n{start_time_srt} --> {end_time_srt}\n{text}\n\n"

    # Remove the trailing newline after the last subtitle (to avoid extra blank line)
    srt_transcription = srt_transcription.strip()

    # Save the SRT transcription to a file
    transcription_path = f"{os.path.splitext(audio_path)[0]}_transcription.srt"
    with open(transcription_path, "w",encoding="utf-8") as f:
        f.write(srt_transcription)  # Write without extra newlines
    
    print(f"SRT transcription generated with Whisper for {audio_path}.")
    return transcription_path

In [4]:
def download_audio_and_transcription(query, num_videos=1, language="en"):
    # Initialize Whisper model and check for CUDA support
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    model = whisper.load_model("base", device=device)
    
    # Search for the video
    search = Search(query)
    videos = search.videos  # Use .videos instead of .results
    if not videos:
        print("No videos found for query.")
        return
    
    # Process the first N videos
    for i, video in enumerate(videos[:num_videos]):
        video_url = video.watch_url
        #video_title = video.title.replace(" ", "_")
        # Remove all invalid characters in a file name using regular expressions
        video_title = re.sub(r"[^\w]", "_", video.title)

        # Download the audio
        audio_path = f"{video_title}.mp3"
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': f"{video_title}.%(ext)s",  # Adjust the template to avoid double extensions
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': True,
        }
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"Downloading audio for {video_title}...")
            ydl.download([video_url])
        
        # Check for YouTube transcription
        captions = video.captions
        if captions:
            for lang in captions:
                if lang.code == language:
                    transcript = lang.generate_srt_captions()
                    transcription_path = f"{video_title}_transcription.srt"
                    with open(transcription_path, "w",encoding="utf-8") as f:
                        f.write(transcript)
                    print(f"Downloaded YouTube transcription for {video_title}.")
                    break
            else:
                print(f"No English transcription available for {video_title}, generating with Whisper...")
                transcription_path = transcribe_with_whisper(audio_path, model, language)
        else:
            print(f"No transcription available for {video_title}, generating with Whisper...")
            transcription_path = transcribe_with_whisper(audio_path, model, language)
        
        print(f"Audio and transcription saved for {video_title}.")
    
    print(f"Processed {num_videos} videos.")

In [5]:
# Usage
query = "دوس اونلاين"
num_videos = 5  # Modify this number to download and transcribe N videos
language = "ar"  # Language code for YouTube transcription (if available)
download_audio_and_transcription(query, num_videos, language)

Using device: cuda


d:\prog apps\python\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


No English transcription available for هذا_الكتاب_خسرني_مليون_جنيه, generating with Whisper...
 الكتابة خصرني مليون جنيه
 هذا الكتاب
 لما تبط الحاجات اللي فيي خصيرت مليون جنيه
 حياه
 هو رك مأجبر من كده ولكن
 شندل عن وان
 يعني دي أحد العنوين الوحدية تمانا نوك لك بيت أو حاجة مش حاية
 ولكن أسف هو حي
 و الكتابة برضو تسبب النفيشة في المرحلة الإعدادية هو وصغضيكه
 حربه من البيت
 سعيا مراء تحكيك ما يعد به
 هذا الكتاب وهربا من التعليم التقليد والواضيف التقليدية
 اتابريش شد برضات لروبر تكيوسكي
 برضو تكيوسكي
 من أشهر الكتابة بكليم عن الاستثمار
 والإدخار والوصول
 والخصول
 والحاجات دي كلها
 إذ لم يكن الأشخر في العالم
 كتاب رشحناة وبنرشحه في الحلقات اللي كنت بنعملها عن الكتب ومراجعات الكتب
 ولكن لأصدقاء به هجمو الكتاب والكاتب ومختلفين معه
 و بهجمو الكتابة كتيرة جداً موجودة في كتاب
 فش هاجديدة كل حاجة مكتلاف جوغيل وفمكالات
 و بشكل محطة والكتاب والدامك
 من غير ما تحتاج
 يعني أنت تأيروا كتاب
 ما بعطب رش إنك لمحي
 و نصب مام في المي
 على طول أول ما عرف النوع لألقاء الموضوع
 الشخص الوحيد
 اللي هو هايب أغن

KeyboardInterrupt: 